In [1]:
%discover
%connect test-stm32 serial

test-esp32                     serial:///dev/cu.usbserial-0160B5B8     
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
test-argon                     serial:///dev/cu.usbmodem1413101        
test-samd                      serial:///dev/cu.usbmodem1413201        
test-stm32                     serial:///dev/cu.usbmodem208E307542521  
Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521


In [1]:
%connect test-stm32
%softreset
import start_urpc
from urpc import *

Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [1]:
import sys
sys_ = import_('sys')

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))

Platforms:
   host:    pyboard
   server:  esp32


In [ ]:
# %connect test-stm32 serial
%connect test-esp32 serial
%softreset

if False:
    import start_urpc
    from urpc import *


import socket
import network
import uos
import errno
from uio import IOBase 

last_client_socket = None
server_socket = None

import io
messages = io.StringIO()

# Provide necessary functions for dupterm and replace telnet control characters that come in.
class TelnetWrapper(IOBase):
    def __init__(self, socket):
        self.socket = socket
        self.discard_count = 0
        
    def readinto(self, b):
        readbytes = 0
        for i in range(len(b)):
            try:
                byte = 0
                # discard telnet control characters and
                # null bytes 
                while(byte == 0):
                    byte = self.socket.recv(1)[0]
                    if byte == 0xFF:
                        self.discard_count = 2
                        byte = 0
                    elif self.discard_count > 0:
                        self.discard_count -= 1
                        byte = 0
                    
                b[i] = byte
                
                readbytes += 1
            except (IndexError, OSError) as e:
                if type(e) == IndexError or len(e.args) > 0 and e.args[0] == errno.EAGAIN:
                    if readbytes == 0:
                        return None
                    else:
                        return readbytes
                else:
                    messages.write("*** readinto {}\n".format(e))
                    raise
        return readbytes
    
    def write(self, data):
        # we need to write all the data but it's a non-blocking socket
        # so loop until it's all written eating EAGAIN exceptions
        while len(data) > 0:
            try:
                written_bytes = self.socket.write(data)
                data = data[written_bytes:]
            except OSError as e:
                if len(e.args) > 0 and e.args[0] == errno.EAGAIN:
                    # can't write yet, try again
                    pass
                else:
                    # something else...propagate the exception
                    messages.write("*** write {}\n".format(e))
                    raise
    
    def close(self):
        self.socket.close()

# Attach new clients to dupterm and 
# send telnet control characters to disable line mode
# and stop local echoing
def accept_telnet_connect(telnet_server):
    global last_client_socket
    
    if last_client_socket:
        # close any previous clients
        uos.dupterm(None)
        last_client_socket.close()
    
    last_client_socket, remote_addr = telnet_server.accept()
    print("Telnet connection from:", remote_addr)
    last_client_socket.setblocking(False)
    # dupterm_notify() not available under MicroPython v1.1
    if hasattr(uos, 'dupterm_notify'):
        print("notify!")
        last_client_socket.setsockopt(socket.SOL_SOCKET, 20, uos.dupterm_notify)
    
    last_client_socket.sendall(bytes([255, 252, 34])) # don't allow line mode
    last_client_socket.sendall(bytes([255, 251, 1]))  # turn off local echo
    
    print("attach dupterm")
    uos.dupterm(TelnetWrapper(last_client_socket))
    print("return from accept")

def stop():
    global server_socket, last_client_socket
    uos.dupterm(None)
    if server_socket:
        server_socket.close()
    if last_client_socket:
        last_client_socket.close()

# start listening for telnet connections on port 23
def start(port=23):
    print("start ...")
    stop()
    global server_socket
    print("server_socket ...")
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    print("getaddrinfo")
    ai = socket.getaddrinfo("0.0.0.0", port)
    addr = ai[0][4]
    print("addr", addr)
    
    server_socket.bind(addr)
    print("listen")
    server_socket.listen(1)
    
    print("url ...")
    for i in (network.AP_IF, network.STA_IF):
        wlan = network.WLAN(i)
        if wlan.active():
            print("Telnet server started on telnet://{}:{}".format(wlan.ifconfig()[0], port))
            
    # server_socket.setsockopt(socket.SOL_SOCKET, 20, accept_telnet_connect)
    if True:
        print("accept ...")
        accept_telnet_connect(server_socket)
        print("messages:", messages.getvalue())

while True:
    try:
        start()
    except KeyboardInterrupt:
        print("KBI")
    import time
    print("sleep ...")
    time.sleep(2)
print("DONE!")

Connected to test-esp32 @ serial:///dev/cu.usbserial-0160B5B8

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

start ...
server_socket ...
getaddrinfo
addr ('0.0.0.0', 23)
listen
url ...
Telnet server started on telnet://10.39.40.135:23
accept ...
Telnet connection from: ('10.39.40.114', 61915)
notify!
attach dupterm
return from accept
messages: 
sleep ...
start ...
server_socket ...
getaddrinfo
addr ('0.0.0.0', 23)
listen
url ...
Telnet server started on telnet://10.39.40.135:23
accept ...


Interrupted

b"KBI\r\nsleep ...\r\nstart ...\r\nserver_socket ...\r\ngetaddrinfo\r\naddr ('0.0.0.0', 23)\r\nlisten\r\nurl ...\r\nTelnet server started on telnet://10.39.40.135:23\r\naccept ...\r\n"


In [ ]:
start()

import machine
def blink():
    led = machine.Pin(13, mode=machine.Pin.OUT)
    while True:
        led.on()
        time.sleep(0.1)
        led.off()
        time.sleep(1.0)

blink()

In [1]:
print(messages.getvalue())

b'notify!\nattach dupterm!\naccept ...\r\nnotify!\nattach dupterm!\naccept ...\r\n'
